In [1]:
import numpy as np
from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt





In [3]:
# ——————————————————————————————————————————————
# Assumed pre-existing plotting style dict:
# rc_default = { … as you defined … }
# plt.rcParams.update(rc_default)
# plt.rcParams['text.usetex'] = True

# ——————————————————————————————————————————————
# 1) Load the FITS tables
gaia_cat= '/its/home/bb345/5-4most_data/other_data/gaia_sources/gaia-mask-dr10_bg_foot.fits'      # must contain RA, DEC, G, R_medium_arcsec
bg_cat=   '/its/home/bb345/5-4most_data/CRS/target_catalogues/BG/full_legacy_no_colour_sel/reduced/desi_bg_nomaskbit_mask_4M_reduced_columns.fits'  # must contain RA, DEC




In [4]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u
from scipy.spatial import KDTree # For Method 2

# Your default plotting settings
rc_default = {}
rc_default['font.family'] = 'serif'
rc_default['font.size'] = 18 # Reduced slightly for better fit with multiple plots
rc_default['axes.labelsize'] = 22 # Adjusted
rc_default['axes.labelweight'] = 'normal'
rc_default['axes.linewidth'] = 1.0
rc_default['axes.titlesize'] = 22 # Adjusted
rc_default['xtick.labelsize'] = 20 # Adjusted
rc_default['ytick.labelsize'] = 20 # Adjusted
rc_default['legend.fontsize'] = 18 # Adjusted
rc_default['figure.titlesize'] = 22 # Adjusted
rc_default['savefig.dpi'] = 100

rc_default['xtick.major.size'] = 10
rc_default['xtick.major.width'] = 1
rc_default['xtick.minor.size'] = 10 # Typically minor is smaller than major
rc_default['xtick.minor.width'] = 0 # Typically minor is thinner or not shown

# Latex related
rc_default['text.usetex'] = True
# rc_default['mathtext.fontset'] = 'custom'
# rc_default['mathtext.rm'] = 'Bitstream Vera Sans'
# rc_default['mathtext.it'] = 'Bitstream Vera Sans:italic'
# rc_default['mathtext.bf'] = 'Bitstream Vera Sans:bold'

plt.rcParams.update(rc_default)
plt.style.use('tableau-colorblind10')

# Function to calculate masking radius in degrees (as provided by you)
def mask_radius_for_mag(mag):
    """
    Returns a masking radius in degrees for a star of the given magnitude.
    Used for Tycho-2 and Gaia stars.
    This is in degrees, and is from Rongpu in the thread [decam-chatter 12099].
    """
    return 1630./3600. * 1.396**(-mag)

# --- Configuration Parameters ---
# These would typically be set based on your data and desired plot characteristics
HIST_RANGE_ARCSEC = 200.0  # Half-width of the 2D histogram in arcsec
NBINS_HIST = 100          # Number of bins along each axis of the 2D histogram
BG_ANNULUS_ARCSEC = (100.0, 150.0) # Min and max radii for background estimation

# Placeholder for FITS file paths
# You will need to change these to your actual file paths
GAIA_CAT_PATH = gaia_cat
BG_CAT_PATH = gaia_cat

In [12]:


# Dummy load_catalogues for standalone testing - replace with your actual function
def load_catalogues():
    global GAIA_CAT_PATH, BG_CAT_PATH
    GAIA_CAT_PATH = 'dummy_gaia_catalogue.fits' # Actual paths for your data
    BG_CAT_PATH = 'dummy_bright_galaxy_catalogue.fits' # Actual paths for your data
    print(f"Attempting to load Gaia from: {GAIA_CAT_PATH}")
    print(f"Attempting to load BG from: {BG_CAT_PATH}")
    try:
        # Create dummy tables if files don't exist, for testing flow
        if not hasattr(load_catalogues, 'dummy_gaia_loaded'):
            gaia_ra = np.random.uniform(0, 360, 10000) # Increased size for testing
            gaia_dec = np.random.uniform(-90, 90, 10000)
            gaia_g = np.random.uniform(7, 17, 10000)
            # Add some NaNs/Infs to G to test filtering
            gaia_g[np.random.choice(len(gaia_g), size=10, replace=False)] = np.nan
            gaia_g[np.random.choice(len(gaia_g), size=10, replace=False)] = np.inf
            load_catalogues.dummy_gaia = Table([gaia_ra, gaia_dec, gaia_g], names=('RA', 'DEC', 'G'))
            print("Created dummy Gaia catalogue.")
            load_catalogues.dummy_gaia_loaded = True
        
        if not hasattr(load_catalogues, 'dummy_bg_loaded'):
            bg_ra = np.random.uniform(0, 360, 50000) # Increased size for testing
            bg_dec = np.random.uniform(-90, 90, 50000)
            load_catalogues.dummy_bg = Table([bg_ra, bg_dec], names=('RA', 'DEC'))
            print("Created dummy BG catalogue.")
            load_catalogues.dummy_bg_loaded = True
        
        return load_catalogues.dummy_gaia, load_catalogues.dummy_bg

    except Exception as e:
        print(f"Error loading catalogues: {e}")
        return None, None


def process_density_astropy(gaia_cat, bg_cat, magnitude_bins):
    """
    Computes and visualises BG density around Gaia stars using Astropy's search_around_sky.
    (Version with explicit loop for cross-matching to work around persistent seplimit error)
    """
    if gaia_cat is None or bg_cat is None or len(gaia_cat) == 0 or len(bg_cat) == 0:
        print("Catalogues not loaded or empty. Aborting Astropy processing.")
        return

    # Ensure RA/DEC columns are NumPy arrays before creating SkyCoord
    bg_coords = SkyCoord(np.asarray(bg_cat['RA'])*u.deg, 
                         np.asarray(bg_cat['DEC'])*u.deg, 
                         frame='icrs')

    for i, (g_min, g_max) in enumerate(magnitude_bins):
        print(f"\nProcessing Gaia magnitude bin: {g_min} < G < {g_max}")

        gaia_mask = (gaia_cat['G'] > g_min) & \
                    (gaia_cat['G'] < g_max) & \
                    np.isfinite(gaia_cat['G']) & \
                    np.isfinite(gaia_cat['RA']) & \
                    np.isfinite(gaia_cat['DEC'])
        gaia_subset = gaia_cat[gaia_mask]
        
        avg_mask_radius_arcsec_this_bin = np.nan # Default

        if len(gaia_subset) == 0:
            print(f"No valid Gaia stars found in magnitude bin {g_min} < G < {g_max}.")
        else:
            print(f"Found {len(gaia_subset)} Gaia stars in this bin.")
            gaia_g_magnitudes_np = np.asarray(gaia_subset['G'])
            mask_radii_deg_np = mask_radius_for_mag(gaia_g_magnitudes_np)
            avg_mask_radius_arcsec_this_bin = np.nanmean(mask_radii_deg_np) * 3600.0 # Use nanmean
            if np.isnan(avg_mask_radius_arcsec_this_bin): # if all mask_radii were nan (unlikely if G is finite)
                print("Average mask radius is NaN. Check G magnitudes and mask_radius_for_mag function.")
            else:
                print(f"Average mask radius for this bin: {avg_mask_radius_arcsec_this_bin:.2f} arcsec")
        
        gaia_subset_coords = SkyCoord(np.asarray(gaia_subset['RA'])*u.deg, 
                                      np.asarray(gaia_subset['DEC'])*u.deg, 
                                      frame='icrs')
        
        # Prepare seplimit_quantity for the loop (or if vectorized call was used)
        if len(gaia_subset) > 0: # Should match len(gaia_subset_coords)
             seplimit_quantity_array = mask_radii_deg_np * u.deg
        else:
             seplimit_quantity_array = u.Quantity([], unit=u.deg)

        # --- Explicit loop for search_around_sky ---
        print("Performing cross-match with explicit loop...")
        list_idx_gaia_original = [] # Stores original index from gaia_subset
        list_idx_bg = []
        list_sep2d_angles = [] 

        if len(gaia_subset_coords) > 0: # Only loop if there are Gaia stars in the subset
            for i_g, one_gaia_coord in enumerate(gaia_subset_coords):
                # one_gaia_coord is a scalar SkyCoord
                # Get the specific seplimit for this star
                one_seplimit = seplimit_quantity_array[i_g] # This is a scalar Quantity
                
                # Call: array.search_around_sky(scalar_coord, scalar_seplimit)
                idx_bg_for_one_gaia, _, sep2d_for_one_gaia, _ = \
                    bg_coords.search_around_sky(one_gaia_coord, one_seplimit)
                
                for i_b_match, S_val_match in zip(idx_bg_for_one_gaia, sep2d_for_one_gaia):
                    list_idx_gaia_original.append(i_g) 
                    list_idx_bg.append(i_b_match)
                    list_sep2d_angles.append(S_val_match)
            print(f"Loop completed. Found {len(list_idx_gaia_original)} pairs.")

        if list_idx_gaia_original: # If any pairs were found
            idx_gaia = np.array(list_idx_gaia_original) # Indices relative to gaia_subset
            idx_bg = np.array(list_idx_bg)       # Indices relative to bg_cat
            # Convert list of Angle objects to a single Quantity array
            sep2d_values_deg = [s.to_value(u.deg) for s in list_sep2d_angles]
            sep2d = u.Quantity(sep2d_values_deg, unit=u.deg) 
        else: # No pairs found or gaia_subset_coords was empty
            idx_gaia = np.array([], dtype=int)
            idx_bg = np.array([], dtype=int)
            sep2d = u.Quantity([], unit=u.deg)
        # --- End of explicit loop section ---

        if len(idx_gaia) == 0: # Check if any pairs were actually found and processed
            print("No BG sources found around Gaia stars in this bin (after explicit loop).")
            # Plotting for no pairs (same as before)
            fig, ax = plt.subplots(figsize=(8, 8))
            ax.set_title(rf"BG Overdensity: ${g_min} < G < {g_max}$ (No Pairs)")
            ax.set_xlabel(r'$\Delta \alpha \cos \delta$ (arcsec)')
            ax.set_ylabel(r'$\Delta \delta$ (arcsec)')
            ax.set_xlim(-HIST_RANGE_ARCSEC, HIST_RANGE_ARCSEC)
            ax.set_ylim(-HIST_RANGE_ARCSEC, HIST_RANGE_ARCSEC)
            ax.text(0.5, 0.5, "No BG-Gaia pairs found", ha='center', va='center', transform=ax.transAxes)
            if not np.isnan(avg_mask_radius_arcsec_this_bin) and avg_mask_radius_arcsec_this_bin > 0:
                mask_circle = plt.Circle((0, 0), avg_mask_radius_arcsec_this_bin, color='red', fill=False, 
                                         linestyle='--', linewidth=2, label=f'Avg. $R_{{mask}}$ ({avg_mask_radius_arcsec_this_bin:.1f} arcsec)')
                ax.add_artist(mask_circle)
                ax.legend(loc='upper right')
            ax.set_aspect('equal', adjustable='box')
            plt.tight_layout()
            plt.savefig(f"bg_overdensity_astropy_looped_mag_bin_{i+1}_no_pairs.png")
            plt.close(fig)
            continue

        # Calculate projected separations in arcseconds
        # Matched Gaia stars: gaia_subset[idx_gaia] (idx_gaia are indices for gaia_subset)
        # Matched BG sources: bg_cat[idx_bg]
        
        delta_ra_deg_all = (np.asarray(bg_cat['RA'][idx_bg]) - np.asarray(gaia_subset['RA'][idx_gaia]))
        delta_dec_deg_all = (np.asarray(bg_cat['DEC'][idx_bg]) - np.asarray(gaia_subset['DEC'][idx_gaia]))
        
        delta_ra_deg_all[delta_ra_deg_all > 180] -= 360
        delta_ra_deg_all[delta_ra_deg_all < -180] += 360
        
        delta_ra_all = delta_ra_deg_all * u.deg
        delta_dec_all = delta_dec_deg_all * u.deg
        
        cos_dec_gaia_matched = np.cos(np.asarray(gaia_subset['DEC'][idx_gaia]) * u.deg.to(u.rad))
        
        delta_x_arcsec = (delta_ra_all * cos_dec_gaia_matched).to(u.arcsec).value
        delta_y_arcsec = delta_dec_all.to(u.arcsec).value

        # Create 2D histogram (same as before)
        bins = np.linspace(-HIST_RANGE_ARCSEC, HIST_RANGE_ARCSEC, NBINS_HIST + 1)
        hist_2d, x_edges, y_edges = np.histogram2d(
            delta_x_arcsec, delta_y_arcsec, bins=(bins, bins)
        )

        # Background estimation (same as before)
        x_centers = (x_edges[:-1] + x_edges[1:]) / 2
        y_centers = (y_edges[:-1] + y_edges[1:]) / 2
        radius_map_check = np.sqrt(x_centers[:, np.newaxis]**2 + y_centers[np.newaxis, :]**2)
        
        bg_mask = (radius_map_check >= BG_ANNULUS_ARCSEC[0]) & (radius_map_check <= BG_ANNULUS_ARCSEC[1])
        
        h_mean_bg = 1e-9 # Default
        if np.sum(bg_mask) > 0:
            mean_val = np.mean(hist_2d[bg_mask])
            if mean_val > 0: # Ensure mean background is positive
                h_mean_bg = mean_val
            else:
                print(f"Warning: Mean background count in annulus is <= 0 ({mean_val:.4f}). Using default 1e-9.")
        else:
            print("Warning: Background annulus is empty or too small. Using default H_mean_bg = 1e-9.")
            
        print(f"Mean background count per bin (H_mean_bg): {h_mean_bg:.4f}")

        epsilon = 1e-9 
        hist_norm = np.log2(np.maximum(hist_2d, epsilon) / h_mean_bg) 
        
        vmin_plot, vmax_plot = -2, 3 

        fig, ax = plt.subplots(figsize=(9, 8)) 
        im = ax.pcolormesh(x_edges, y_edges, hist_norm.T, cmap='coolwarm', vmin=vmin_plot, vmax=vmax_plot)
        
        ax.set_title(rf"BG Overdensity: ${g_min} < G < {g_max}$ (Looped AstroPy)")
        ax.set_xlabel(r'$\Delta \alpha \cos \delta$ (arcsec)')
        ax.set_ylabel(r'$\Delta \delta$ (arcsec)')
        
        cbar = fig.colorbar(im, ax=ax, label=r'$\log_2 (H / \bar{H}_{bg})$')
        
        if not np.isnan(avg_mask_radius_arcsec_this_bin) and avg_mask_radius_arcsec_this_bin > 0:
            mask_circle = plt.Circle((0, 0), avg_mask_radius_arcsec_this_bin, color='lime', fill=False, 
                                     linestyle='--', linewidth=2, label=rf'Avg. $R_{{mask}}$ ({avg_mask_radius_arcsec_this_bin:.1f} arcsec)')
            ax.add_artist(mask_circle)
            ax.legend(loc='upper right')
        
        ax.set_aspect('equal', adjustable='box')
        ax.set_xlim(-HIST_RANGE_ARCSEC, HIST_RANGE_ARCSEC)
        ax.set_ylim(-HIST_RANGE_ARCSEC, HIST_RANGE_ARCSEC)
        
        plt.tight_layout()
        plt.savefig(f"bg_overdensity_astropy_looped_mag_bin_{i+1}.png") # Note suffix change
        print(f"Plot saved: bg_overdensity_astropy_looped_mag_bin_{i+1}.png")
        plt.close(fig)

# --- Main execution block (example) ---
if __name__ == '__main__':
    print("Running Method 1: Astropy (Looped search_around_sky)")
    
    try:
        import astropy
        print(f"Astropy version: {astropy.__version__}")
    except ImportError:
        print("Astropy not found.")

    magnitude_bins_main = [
        (8, 11),
        (11, 13),
        (13, 16)
    ]
    
gaia_data, bg_data = load_catalogues() # Uses dummy data if actual files not found

# Additional filtering for catalogue validity
if gaia_data is not None and len(gaia_data) > 0:
    valid_gaia_rows = np.isfinite(np.asarray(gaia_data['RA'])) & \
                        np.isfinite(np.asarray(gaia_data['DEC'])) & \
                        np.isfinite(np.asarray(gaia_data['G']))
    gaia_data = gaia_data[valid_gaia_rows]
    print(f"Filtered Gaia catalogue to {len(gaia_data)} valid rows.")

if bg_data is not None and len(bg_data) > 0:
    valid_bg_rows = np.isfinite(np.asarray(bg_data['RA'])) & \
                    np.isfinite(np.asarray(bg_data['DEC']))
    bg_data = bg_data[valid_bg_rows]
    print(f"Filtered BG catalogue to {len(bg_data)} valid rows.")

process_density_astropy(gaia_data, bg_data, magnitude_bins_main)

Running Method 1: Astropy (Looped search_around_sky)
Astropy version: 7.0.0
Attempting to load Gaia from: dummy_gaia_catalogue.fits
Attempting to load BG from: dummy_bright_galaxy_catalogue.fits
Created dummy Gaia catalogue.
Created dummy BG catalogue.
Filtered Gaia catalogue to 9980 valid rows.
Filtered BG catalogue to 50000 valid rows.

Processing Gaia magnitude bin: 8 < G < 11
Found 2965 Gaia stars in this bin.
Average mask radius for this bin: 70.88 arcsec
Performing cross-match with explicit loop...


ValueError: One of the inputs to search_around_sky is a scalar. search_around_sky is intended for use with array coordinates, not scalars.  Instead, use ``coord1.separation(coord2) < seplimit`` to find the coordinates near a scalar coordinate.

In [13]:
def skycoord_to_cartesian(coords):
    """Converts Astropy SkyCoord to 3D Cartesian unit vectors."""
    return coords.cartesian.xyz.value.T # Transpose to get (N, 3) array

def process_density_kdtree(gaia_cat, bg_cat, magnitude_bins):
    """
    Computes and visualises BG density around Gaia stars using KDTree.
    """
    if gaia_cat is None or bg_cat is None:
        print("Catalogues not loaded. Aborting.")
        return

    # Convert BG catalogue to 3D Cartesian and build KDTree
    bg_skycoords = SkyCoord(bg_cat['RA']*u.deg, bg_cat['DEC']*u.deg, frame='icrs')
    bg_cartesian = skycoord_to_cartesian(bg_skycoords)
    if bg_cartesian.shape[0] == 0:
        print("BG catalogue is empty after coordinate conversion.")
        return
    bg_kdtree = KDTree(bg_cartesian)

    all_delta_x_arcsec = []
    all_delta_y_arcsec = []
    
    # Store average mask radii for plotting circles later, if no pairs are found in a bin
    avg_mask_radii_bins = []


    for i, (g_min, g_max) in enumerate(magnitude_bins):
        print(f"\nProcessing Gaia magnitude bin: {g_min} < G < {g_max} (KDTree Method)")

        # Filter Gaia stars by magnitude
        gaia_subset = gaia_cat[(gaia_cat['G'] > g_min) & (gaia_cat['G'] < g_max)]
        if not gaia_subset:
            print(f"No Gaia stars found in magnitude bin {g_min} < G < {g_max}.")
            avg_mask_radii_bins.append(0) # Placeholder
            # Create empty plot for this bin
            fig, ax = plt.subplots(figsize=(8, 8))
            ax.set_title(rf"BG Overdensity: ${g_min} < G < {g_max}$ (KDTree - No Gaia Stars)")
            ax.set_xlabel(r'$\Delta \alpha \cos \delta$ (arcsec)')
            ax.set_ylabel(r'$\Delta \delta$ (arcsec)')
            ax.set_xlim(-HIST_RANGE_ARCSEC, HIST_RANGE_ARCSEC)
            ax.set_ylim(-HIST_RANGE_ARCSEC, HIST_RANGE_ARCSEC)
            ax.text(0.5, 0.5, "No Gaia stars in bin", ha='center', va='center', transform=ax.transAxes)
            ax.set_aspect('equal', adjustable='box')
            plt.tight_layout()
            plt.savefig(f"bg_overdensity_kdtree_mag_bin_{i+1}_no_gaia.png")
            plt.close(fig)
            continue
        
        print(f"Found {len(gaia_subset)} Gaia stars in this bin.")

        gaia_subset_skycoords = SkyCoord(gaia_subset['RA']*u.deg, gaia_subset['DEC']*u.deg, frame='icrs')
        gaia_subset_cartesian = skycoord_to_cartesian(gaia_subset_skycoords)
        
        mask_radii_deg = mask_radius_for_mag(gaia_subset['G'])
        avg_mask_radius_arcsec_this_bin = np.mean(mask_radii_deg) * 3600.0
        avg_mask_radii_bins.append(avg_mask_radius_arcsec_this_bin)
        print(f"Average mask radius for this bin: {avg_mask_radius_arcsec_this_bin:.2f} arcsec")

        # Convert angular search radii to 3D chord distances
        mask_radii_rad = mask_radii_deg * u.deg.to(u.rad)
        chord_distances = 2 * np.sin(mask_radii_rad / 2.0)

        bin_delta_x_arcsec = []
        bin_delta_y_arcsec = []

        # Loop over Gaia stars (vectorisation here is harder due to star-specific radius with KDTree query_ball_point)
        for j in range(len(gaia_subset)):
            gaia_star_coord = gaia_subset_skycoords[j]
            gaia_star_cartesian = gaia_subset_cartesian[j]
            search_radius_chord = chord_distances[j]
            
            # Query KDTree for BG sources within the chord distance
            # This returns indices relative to bg_cartesian (and thus bg_skycoords/bg_cat)
            indices_bg_candidates = bg_kdtree.query_ball_point(gaia_star_cartesian, r=search_radius_chord)
            
            if not indices_bg_candidates:
                continue

            bg_candidates_coords = bg_skycoords[indices_bg_candidates]
            
            # Precise angular separation check
            separations = gaia_star_coord.separation(bg_candidates_coords)
            
            # Ensure we use the correct star-specific mask radius for THIS star
            actual_mask_radius_deg_this_star = mask_radii_deg[j]
            valid_indices = np.where(separations.deg <= actual_mask_radius_deg_this_star)[0]
            
            if not len(valid_indices):
                continue
            
            # Get original indices of confirmed BG sources
            confirmed_bg_original_indices = np.array(indices_bg_candidates)[valid_indices]

            # Calculate projected separations for confirmed pairs
            delta_ra = (bg_cat['RA'][confirmed_bg_original_indices] - gaia_subset['RA'][j]) * u.deg
            delta_dec = (bg_cat['DEC'][confirmed_bg_original_indices] - gaia_subset['DEC'][j]) * u.deg
            cos_dec_gaia = np.cos(gaia_subset['DEC'][j] * u.deg.to(u.rad))
            
            dx = (delta_ra * cos_dec_gaia).to(u.arcsec).value
            dy = delta_dec.to(u.arcsec).value
            
            bin_delta_x_arcsec.extend(dx)
            bin_delta_y_arcsec.extend(dy)

        if not bin_delta_x_arcsec:
            print("No BG sources found around Gaia stars in this bin using KDTree method.")
             # Create an empty plot for consistency
            fig, ax = plt.subplots(figsize=(8, 8))
            ax.set_title(rf"BG Overdensity: ${g_min} < G < {g_max}$ (KDTree - No Pairs)")
            ax.set_xlabel(r'$\Delta \alpha \cos \delta$ (arcsec)')
            ax.set_ylabel(r'$\Delta \delta$ (arcsec)')
            ax.set_xlim(-HIST_RANGE_ARCSEC, HIST_RANGE_ARCSEC)
            ax.set_ylim(-HIST_RANGE_ARCSEC, HIST_RANGE_ARCSEC)
            ax.text(0.5, 0.5, "No BG-Gaia pairs found", ha='center', va='center', transform=ax.transAxes)
             # Draw average masking radius circle
            mask_circle = plt.Circle((0, 0), avg_mask_radius_arcsec_this_bin, color='red', fill=False, 
                                     linestyle='--', linewidth=2, label=f'Avg. $R_{{mask}}$ ({avg_mask_radius_arcsec_this_bin:.1f} arcsec)')
            ax.add_artist(mask_circle)
            ax.legend(loc='upper right')
            ax.set_aspect('equal', adjustable='box')
            plt.tight_layout()
            plt.savefig(f"bg_overdensity_kdtree_mag_bin_{i+1}_no_pairs.png")
            plt.close(fig)
            continue

        # Create 2D histogram for this bin
        bins = np.linspace(-HIST_RANGE_ARCSEC, HIST_RANGE_ARCSEC, NBINS_HIST + 1)
        hist_2d, x_edges, y_edges = np.histogram2d(
            np.array(bin_delta_x_arcsec), np.array(bin_delta_y_arcsec), bins=(bins, bins)
        )

        # Background estimation (same as in Astropy method)
        x_centers = (x_edges[:-1] + x_edges[1:]) / 2
        y_centers = (y_edges[:-1] + y_edges[1:]) / 2
        radius_map = np.sqrt(x_centers[:, np.newaxis]**2 + y_centers[np.newaxis, :]**2)
        
        bg_mask = (radius_map >= BG_ANNULUS_ARCSEC[0]) & (radius_map <= BG_ANNULUS_ARCSEC[1])
        
        if np.sum(bg_mask) > 0:
            h_mean_bg = np.mean(hist_2d[bg_mask])
        else:
            print("Warning: Background annulus is empty. Setting H_mean_bg to 1e-9.")
            h_mean_bg = 1e-9
            
        print(f"Mean background count per bin (H_mean_bg): {h_mean_bg:.4f}")
        
        epsilon = 1e-9
        hist_norm = np.log2(np.maximum(hist_2d, epsilon) / np.maximum(h_mean_bg, epsilon))
        
        vmin_plot, vmax_plot = -2, 3 # Consistent plot scale

        # Plotting
        fig, ax = plt.subplots(figsize=(9, 8))
        im = ax.pcolormesh(x_edges, y_edges, hist_norm.T, cmap='coolwarm', vmin=vmin_plot, vmax=vmax_plot)
        
        ax.set_title(rf"BG Overdensity: ${g_min} < G < {g_max}$ (KDTree)")
        ax.set_xlabel(r'$\Delta \alpha \cos \delta$ (arcsec)')
        ax.set_ylabel(r'$\Delta \delta$ (arcsec)')
        
        cbar = fig.colorbar(im, ax=ax, label=r'$\log_2 (H / \bar{H}_{bg})$')
        
        mask_circle = plt.Circle((0, 0), avg_mask_radius_arcsec_this_bin, color='lime', fill=False, 
                                 linestyle='--', linewidth=2, label=rf'Avg. $R_{{mask}}$ ({avg_mask_radius_arcsec_this_bin:.1f} arcsec)')
        ax.add_artist(mask_circle)
        
        ax.legend(loc='upper right')
        ax.set_aspect('equal', adjustable='box')
        ax.set_xlim(-HIST_RANGE_ARCSEC, HIST_RANGE_ARCSEC)
        ax.set_ylim(-HIST_RANGE_ARCSEC, HIST_RANGE_ARCSEC)

        plt.tight_layout()
        plt.savefig(f"bg_overdensity_kdtree_mag_bin_{i+1}.png")
        print(f"Plot saved: bg_overdensity_kdtree_mag_bin_{i+1}.png")
        plt.close(fig)



In [14]:
# --- To run the KDTree method ---
print("\nRunning Method 2: KDTree search")
gaia_cat_data, bg_cat_data = load_catalogues() # Reload or reuse if already loaded
if gaia_cat_data and bg_cat_data:
   process_density_kdtree(gaia_cat_data, bg_cat_data, magnitude_bins)


Running Method 2: KDTree search
Attempting to load Gaia from: dummy_gaia_catalogue.fits
Attempting to load BG from: dummy_bright_galaxy_catalogue.fits

Processing Gaia magnitude bin: 8 < G < 11 (KDTree Method)
Found 2965 Gaia stars in this bin.
Average mask radius for this bin: 70.88 arcsec
Mean background count per bin (H_mean_bg): 0.0000
Plot saved: bg_overdensity_kdtree_mag_bin_1.png

Processing Gaia magnitude bin: 11 < G < 13 (KDTree Method)
Found 2043 Gaia stars in this bin.
Average mask radius for this bin: 30.33 arcsec
No BG sources found around Gaia stars in this bin using KDTree method.

Processing Gaia magnitude bin: 13 < G < 16 (KDTree Method)
Found 3018 Gaia stars in this bin.
Average mask radius for this bin: 13.54 arcsec
Mean background count per bin (H_mean_bg): 0.0000
Plot saved: bg_overdensity_kdtree_mag_bin_3.png
